# Contact inhibition functions

This notebook goes through how contact inhibition functions can be used to quantify the number of neighbrs around a cell and decide to stop proliferation based on this value.

In [4]:
import numpy as np
import vedo

from neurorosettes.neurons import Neuron, NeuronFactory
from neurorosettes.clocks import ClocksFactory
from neurorosettes.subcellular import CellBody, ObjectFactory
from neurorosettes.simulation import Container, Timer
from neurorosettes import utilities
from neurorosettes import physics
from neurorosettes.grid import UniformGrid, OneLevelDensityCheck, TwoLevelsDensityCheck


def create_tissue() -> list:
    coordinates = [np.array([x + 8 * (i % 2), y, 0])
                   for x in range(-40, 41, 16)
                   for i, y in enumerate(range(-40, 41, 16))]
    return coordinates

## Creating objects and interactions

In [6]:
# Define the mechanical parameters of the objects
factory = ObjectFactory(
    cell_radius=8.0,
    cell_interaction_factor=1.25,
    neurite_radius=1.0,
    neurite_interaction_factor=2.0,
    neurite_spring_constant=10.0,
    neurite_default_length=15.0,
)

clocks = ClocksFactory(proliferation_rate=0.1, death_rate=0.1, differentiation_rate=0.1)

neuron_factory = NeuronFactory(4, factory, clocks)

# Define the mechanical parameters of the interactions
contact_function = physics.PotentialsFactory(sphere_sphere_adhesion=0.4,
                                             sphere_sphere_repulsion=10.0,
                                             sphere_sphere_smoothness=1,
                                             sphere_cylinder_adhesion=4.0,
                                             sphere_cylinder_repulsion=10.0,
                                             sphere_cylinder_smoothness=1,
                                             cylinder_cylinder_adhesion=4.0,
                                             cylinder_cylinder_repulsion=10.0,
                                             cylinder_cylinder_smoothness=1)

## One-level contact inhibition

The one-level contact inhibition function evaluates the number of neighbors in the current cell of the domain grid, as well as the cells surrounding it. This function evaluates if the number of cell bodies registered in these cells is larger than a user-defined value.

In [9]:
# Initialize simulation objects
container = Container(grid=UniformGrid(*[-70, 70, 20]),
                     simulation_2d=True,
                     neuron_factory=neuron_factory,
                     contact_factory=contact_function)

for position in create_tissue():
    # Populate environment with cells
    neuron = container.create_new_neuron(position)
    neuron.set_outgrowth_axis(utilities.get_random_unit_vector(two_dimensions=True))
    neuron.clocks.set_clocks(0.04, 0.00000001, 0.0000002)
    container.register_neuron(neuron)
    
neuron = container.neurons[29]
container.neurons[29].cell.sphere.c("lightgrey")

cell_neighbors = [neighbor
                  for neighbor in container.grid.get_close_cells(neuron.cell.position)]

for cell in cell_neighbors:
    if cell is not neuron.cell:
        cell.sphere.c("orange")

container.animator.plotter.show(resetcam=False, interactive=False)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)

In [63]:
container = Container(grid=UniformGrid(*[-70, 70, 20]),
                     simulation_2d=True,
                     object_factory=factory,
                     contact_factory=contact_function)

for position in create_tissue():
    # Populate environment with cells
    neuron = container.create_new_neuron(position)
    neuron.set_outgrowth_axis(utilities.get_random_unit_vector(two_dimensions=True))
    neuron.clocks.set_clocks(0.04, 0.00000001, 0.0000002)
    container.register_neuron(neuron)
    
neuron = container.neurons[29]
container.neurons[29].cell.sphere.c("lightblue")

cell_neighbors = [neighbor
                  for neighbor in container.grid.get_close_cells(neuron.cell.position)]

for cell in cell_neighbors:
    if cell is not neuron.cell:
        cell.sphere.c("orange")
        
    if len(cell_neighbors) > 15:

        acceptable_radius = 20
        nearby = [neighbor for neighbor in cell_neighbors
                  if np.linalg.norm(neighbor.position - neuron.cell.position) <= acceptable_radius]

        for n in nearby:
            if n is not neuron.cell:
                n.sphere.c("yellow")

container.animator.plotter.show(resetcam=False, interactive=False)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)

In [56]:
vedo.screenshot("two_level.png")

## Two-level contact inhibition

The two-level contact inhibition function consists of a first level that does the same as the one-level contact inhibition function, plus an additional evaluation. In this second level, the number of neighbors inside a user-defined radius of interest is evaluated, to assess if cells are fully surrounded by other cells or if there is some direction in which growth is possible.

In [5]:
for position in create_tissue():
    # Populate environment with cells
    neuron = container.create_new_neuron(position)
    neuron.set_outgrowth_axis(utilities.get_random_unit_vector(two_dimensions=True))
    neuron.clocks.set_clocks(0.04, 0.00000001, 0.0000002)
    container.register_neuron(neuron)

container.animator.plotter += clock
container.animator.plotter += legend_outline
container.animator.plotter += legend
container.animator.plotter += cycling_legend
container.animator.plotter += cycling_text
container.animator.plotter += blocked_legend
container.animator.plotter += blocked_text
container.animator.plotter += active_legend
container.animator.plotter += active_text
container.animator.plotter += neighbor_legend
container.animator.plotter += neighbor_text
container.animator.plotter += nearby_legend
container.animator.plotter += nearby_text

container.animator.plotter.show(resetcam=False, interactive=False)
blocked_cells = []

# Run and plot simulation
for t in pb.range():
    if t % 5 == 0:
        container.advance_cycles(step)
        container.kill()
        container.differentiate()

        for neuron in container.neurons:
            if not neuron.ready_for_division:
                continue

            cell_neighbors = [neighbor
                              for neighbor in container.grid.get_close_objects(neuron.cell.position)
                              if isinstance(neighbor, CellBody)]

            if len(cell_neighbors) > 15:

                acceptable_radius = neuron.cell.mechanics.radius * 2.1
                nearby = [neighbor for neighbor in cell_neighbors
                          if np.linalg.norm(neighbor.position - neuron.cell.position) <= acceptable_radius]

                if len(nearby) < 7:
                    continue

                neuron.cell.sphere.c("blue")
                for neighbor in cell_neighbors:
                    if neighbor is neuron.cell:
                        continue
                    if neighbor in nearby:
                        neighbor.sphere.c("brown")
                    else:
                        neighbor.sphere.c("yellow")

                container.update_drawings()
                for neighbor in cell_neighbors:
                    if neighbor is neuron.cell:
                        neighbor.sphere.c("black")
                        blocked_cells.append(neighbor)
                    else:
                        if neighbor not in blocked_cells:
                            neighbor.sphere.c("red")
                        else:
                            neighbor.sphere.c("black")

                container.update_drawings()
                neuron.clocks.cycle_clock.division_signal = False
                neuron.clocks.cycle_clock.cycle_block = True

        container.divide()
    container.update_cell_positions()
    container.update_drawings()
    container.animator.plotter.show()

    if t % 50 == 0:
        clock.text(f"Simulation step: {t}")


container.animator.plotter.show(interactive=True)

ViewInteractiveWidget(height=720, layout=Layout(height='auto', width='100%'), width=720)